# Machine Learning - Assignment 1

> Hoang Dang - s3927234

- Trong mỗi section có thể có các mục kiểu "chú ý", "kết luận", ..
- Tham khảo: https://www.kaggle.com/code/alejopaullier/make-your-notebooks-look-better


## Abstract

This study aims to employ machine learning methodologies for the prediction of diabetes onset in individuals utilizing different health metrics and demographic information

---

## Table of Contents

- [1. Problem Formulation](#problem_formulation)
- [2. Exploratory Data Analysis](#EDA)
  - [2.1. Descriptive Statistic](#DS)
  - [2.2. Variable Distributions](#VD)
  - [2.3. Correlation Analysis](#CA)
  - [2.4. Feature Engineering](#FE)
- [3. Evaluation Framework](#evaluation_framework)
- [4. Modelling](#modelling)
  - [4.1. Models Proposal](#model_proposal)
  - [4.2. Models Implementation](#model_implementation)
  - [4.3. Models Evaluation](#model_evaluataion)
- [5. Discussion and Conclusion](#sumup)

## 1. Problem Formulation <a class="anchor" id="problem_formulation"></a>

The first step in developing a model is to formulate the problem in a way that we can apply machine learning. In here, the task is to predict the **presence or absence of diabetes** using various health metrics and demographic information of individuals.

Based on the problem type, the training dataset, and the `code_book.txt`, some information we can get are:
- This is a `Supervised Learning` and specifically, `Binary Classification` problem since the training data contains the `Status` column representing the occurance of diabetes, which is also what we are trying to predict (0 for absence and 1 for presence)
- In the training dataset, there are in total 25 columns, in which:
  - The `Id` column will not be used since it does not contribute to the occurance of diabetes
  - The `Status` column will be used as the target variable for the training process
  - The remaining 23 columns are the attributes and will be used for the data analyzing and training processes
- Data attribute types:
  - There are 8 `ordinal attributes` namely GenHlth (1 - 5), MentHlth (1 - 30), PhysHlth (1 - 30), Age (1 - 13), Education (1 - 6), Income (1 - 8), ExtraMedTest (-199, 199), ExtraAlcoholTest (-199, 199)
  - BMI is the only `continuous attribute`
  - The remaining 14 features are `binary attributes`, which can only be either 0 or 1

## 2. Exploratory Data Analysis <a id="EDA"></a>

In this section, we will work on Exploratory Data Analysis (EDA), a crucial step in the data analysis that allows us to gain insights into the structure, distribution, and relationships of different attributes within our dataset.

### 2.1. Descriptive Statistic <a id="DS"></a>

After loading the training dataset into a `Pandas DataFrame`, we can extract some information using `.shape` and `.describe` methods. Based on `output[2]`, we can see that:
- Our dataset has in total 202,944 rows (records) and 24 columns (23 of which are attributes and the `Status` column as the expected output)
- There are **no missing values** in the dataset since all columns have exactly 202,944 rows
- The percentage of people suffering from High Blood Pressure and High Cholesterol are nearly the same at about 42%
- BMI has values ranging from 12 to 98. The average BMI is 28.38, which is relatively high [ref](https://www.cdc.gov/healthyweight/assessing/bmi/adult_bmi/)
- Most people had Cholesterol Check (96%) and Health Care Coverage (95%)
- There are minor of people suffered from Stroke (4%), Heavy Alcohol Consump (6%), No Doctor because of Cost (8%), Heart Disease and Attack (9%), and Walking Difficulty (17%)
- There are more than a half of people having a healthy lifestyle with Physical Activity at 76%, Fruits Consume at 64%, and Veggies Consume at 81%
- Surprisingly, the statistics of ExtraMedTest and ExtraAlcoholTest are nearly the same
- **Only 18% of people suffered from Diabetes (The Target variable)**, we can then conclude that we are having an `imbalance dataset`


### 2.2. Variable Distributions <a id="VD"></a>

In addition to statistic features provided by `Pandas`, we can also use `Matplotlib` and `Seaborn` to visualize different attributes in the dataset. We begin by plotting the `histogram` to see the data distribution of each attribute [ref](https://www.itl.nist.gov/div898/handbook/eda/section3/histogra.htm) and after that, we use `box plot` to see the variation of information [ref](https://www.itl.nist.gov/div898/handbook/eda/section3/boxplot.htm) 

Using histogram (`output[4]`), we can see that:
- ExtraMedTest and ExtraAlcoholTest are relatively similar to each other and have a **symmetrical distribution**
- BMI is **right skewed** with the majority falls between 20 and 30
- HighBP, HighChol, Smoker, and Sex are **balance**, while CholCheck, Stroke, AnyHealthCare, NoDocbcCost, MentHlth, PhysHlth, DiffWalk, Education, Income, and Status are **imbalance**
- In MentHlth and PhysHlth, the value of 30 gets surprisingly higher compared to previous values close to it. It is possible that someone, in the past 30 days, did not have a good mental/physical health experience, so we can conclude that this is not a data error

On the other hand, with box plot (`output[5]`), we can see that:
- There are a lot of outliers in BMI (from 40 to 100), MentHlth (from 5 to 30), and PhysHlth (from 7 to 30). However, it is possible that someone has BMI up to 100 [ref]() and someone has a 30 days unwell. We can conclude that these outliers are not data collection error and will keep all of them
- Some outliers occuring in **binary attributes** like CholCheck, Stroke, HeartDiseaseorAttack, PhysActivity, Veggies, HvyAlcoholConsump, AnyHealthcare, NoDocbcCost, GenHlth, DiffWalk indicate that there are some imbalance between the two binary classes
- On the other hand, the box plots of HighBP, HighChol, Smoker, Fruits, and Sex show that there is a balance distribution between the values


### 2.3. Correlation Analysis <a id="CA"></a>

In addition to visualizing data distribution, we can also display the correlation between different columns of the dataset. From `output[7]`, we can see that:
- Some high correlated columns are:
  - GenHlth and PhysHlth: 0.53
  - GenHlth and DiffWalk: 0.46
  - DiffWalk and PhysHlth: 0.48
  - GenHlth and Income: -0.37
  - Status and ExtraMedTest: 0.5
  - Status and ExtraAlcoholTest: 0.5
- CholCheck, Smoker, Stroke, Fruits, Veggies, HvyAlcoholConsump, AnyHealthcare, NoDocbcCost, MentHlth, and Sex are low correlated with `Status`


### 2.4. Feature Engineering <a id="FE"></a>

Feature engineering is a crucial step in optimizing the performance of machine learning models. In this section, we delve into several key aspects of feature engineering, including normalization, handling skewed attributes, and addressing duplicates.

- `Normalization`: As our dataset has variables with different value range, we will use **Min-Max scaling** to transform the data to a standardized range between 0 and 1. This ensures that all features contribute equally to the analysis and a faster training process. `output[8]` displays the data distribution after the normalizing process.

- Handling skewed attributes

- Handling duplicates

- About outliers: As stated in previous sections, outliers 

---

- Perform thorough exploratory data analysis to understand the distribution of features, identify correlations, and visualize patterns in the dataset

- Handle missing values, outliers, and perform necessary data preprocessing steps


## 3. Evaluation Framework <a id="evaluation_framework"></a>

- Binary Classification problem -> `Accuracy`, F1, Precision, `Recall`

## 4. Modelling <a id="modelling"></a>

### 4.1. Models Proposal <a id="model_proposal"></a>

- Propose at least three different machine learning models suitable for predicting the status of diabetes development in patients (only use techniques taught in class up to week 5 - inclusive)

- Justify the selection of each model based on its strengths, weaknesses, and suitability for the task


### 4.2. Models Implementation <a id="model_implementation"></a>

- Implement the proposed machine learning models using appropriate libraries (e.g scikit-learn, TensorFlow, Keras, etc.).

- Fine-tune hyperparameters using cross-validation techniques to optimize model performance


### 4.3. Models Evaluataion <a id="model_evaluation"></a>

Submit the prediction for patients in a given test set whose dependent variable (i.e., the presence or absence of diabetes) is hidden


## 5. Discussion and Conclusion <a id="sumup"></a>

- Provide a detailed discussion on the effectiveness of different machine learning models for diabetes prediction.

- Discuss the implications of the findings and potential applications of the predictive models in healthcare settings.

- Highlight areas for future research or improvements in predictive modeling for diabetes

The following describes the dataset columns:

| Attribute             | Description                                                                                                              |
|-----------------------|--------------------------------------------------------------------------------------------------------------------------|
| ID                    | Patient ID                                                                                                               |
| Status                | 0 = no diabetes, 1 = prediabetes or diabetes                                                                             |
| HighBP                | 0 = no high blood pressure, 1 = high blood pressure                                                                      |
| HighChol              | 0 = no high cholesterol, 1 = high cholesterol                                                                            |
| CholCheck             | 0 = no cholesterol check in 5 years, 1 = yes cholesterol check in 5 years                                                |
| BMI                   | Body Mass Index                                                                                                          |
| Smoker                | Have you smoked at least 100 cigarettes in your entire life? (0 = no, 1 = yes)                                           |
| Stroke                | (Ever told) you had a stroke (0 = no, 1 = yes)                                                                          |
| HeartDiseaseorAttack | Coronary heart disease (CHD) or myocardial infarction (MI) (0 = no, 1 = yes)                                             |
| PhysActivity          | Physical activity in the past 30 days (not including job) (0 = no, 1 = yes)                                               |
| Fruits                | Consume fruit 1 or more times per day (0 = no, 1 = yes)                                                                 |
| Veggies               | Consume vegetables 1 or more times per day (0 = no, 1 = yes)                                                            |
| HvyAlcoholConsump     | Heavy drinkers (adult men having more than 14 drinks per week and adult women having more than 7 drinks per week) (0 = no, 1 = yes) |
| AnyHealthcare         | Have any kind of health care coverage, including health insurance, prepaid plans such as HMO, etc. (0 = no, 1 = yes)   |
| NoDocbcCost           | Was there a time in the past 12 months when you needed to see a doctor but could not because of cost? (0 = no, 1 = yes)|
| GenHlth               | Would you say that in general your health is (scale 1-5): 1 = excellent, 2 = very good, 3 = good, 4 = fair, 5 = poor     |
| MentHlth              | Now thinking about your mental health, which includes stress, depression, and problems with emotions, for how many days during the past 30 days was your mental health not good? (scale 1-30 days) |
| PhysHlth              | Now thinking about your physical health, which includes physical illness and injury, for how many days during the past 30 days was your physical health not good? (scale 1-30 days) |
| DiffWalk              | Do you have serious difficulty walking or climbing stairs? (0 = no, 1 = yes)                                            |
| Sex                   | Sex (0 = female, 1 = male)                                                                                              |
| Age                   | 13-level age category: 1 = 18-24, 9 = 60-64, 13 = 80 or older                                                            |
| Education             | Education level (scale 1-6): 1 = Never attended school or only kindergarten, 2 = Grades 1 through 8 (Elementary), 3 = Grades 9 through 11 (Some high school), 4 = Grade 12 or GED (High school graduate), 5 = College 1 year to 3 years (Some college or technical school), 6 = College 4 years or more (College graduate) |
| Income                | Income scale (scale 1-8): 1 = less than $10,000, 5 = less than $35,000, 8 = $75,000 or more                             |
| ExtraMedTest          | The result of an extra medical test, range (-100, 100)                                                                  |
| ExtraAlcoholTest      | The result of an extra alcohol test, range (-100, 100)                                                                  |
